In [ ]:
!pip install unsloth xformers trl peft accelerate bitsandbytes faiss-cpu langchain-community langchain-huggingface sentence-transformers

# RAG

In [ ]:
import gdown

# Скачивание и загрузка клинических рекомендаций
guidelines_id = "1Ctvi5eS39zYDY5paXmIgV9PnVaiRq1UU"
guidelines_name = "russco"
gdown.download(id=guidelines_id, output="guidelines.zip", quiet=False)
!unzip -O CP866 -o -q guidelines.zip -d guidelines

In [ ]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Настройка заголовков для разделения
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

# Инициализируем сплиттеры
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Загрузка документов
loader = DirectoryLoader("./guidelines/"+guidelines_name, glob="./*.md", loader_cls=TextLoader)
documents = loader.load()

final_chunks = []
for doc in documents:
    file_source = doc.metadata.get('source', 'unknown')
    file_name = "".join(os.path.basename(file_source).split(".")[:-1])

    header_splits = markdown_splitter.split_text(doc.page_content)
    for header_chunk in header_splits:
        header_chunk.metadata['source'] = file_name
        sub_chunks = text_splitter.split_documents([header_chunk])
        final_chunks.extend(sub_chunks)

# Загрузка модели эмбеддингов
model_name = "intfloat/multilingual-e5-small"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# 3. Создание векторной базы
vector_db = FAISS.from_documents(final_chunks, embeddings)
vector_db.save_local("russco_faiss")


# Fine-tuning

In [ ]:
import json
import random
import gdown

# Скачивание и загрузка датасета
dataset_id = "1PY_woE3FPjrZcVDKcShVmdCFyBpv_s60"
gdown.download(id=dataset_id, output="full_dset.json", quiet=False)

with open("./full_dset.json", "r") as file:
    data_list = json.load(file)

# Разделение датасета на выборки
random.seed(42)
random.shuffle(data_list)
n_test = round(0.1*len(data_list))
data_list_test, data_list_train = data_list[0:n_test], data_list[n_test:]

# Сохранение выборок
with open("./train_dset.json", 'w', encoding='utf-8') as json_file:
    json.dump(data_list_train, json_file, indent=4, ensure_ascii=False)
with open("./test_dset.json", 'w', encoding='utf-8') as json_file:
    json.dump(data_list_test, json_file, indent=4, ensure_ascii=False)

print(f"\n\nЧисло записей: {len(data_list)}\nРазмер обучающей выборки: {len(data_list_train)}\nРазмер тестовой выборки: {len(data_list_test)}\n")
for key, value in data_list[1].items():
  print(f"===== {key} =====\n{value}\n")

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048

# Загрузка языковой модели
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

# Добавление LoRA адаптеров
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Степень адаптации (чем выше, тем умнее, но тяжелее)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state=42
)

In [ ]:
from datasets import Dataset

# Формирование промптов для дообучения

prompt_style = """Ниже представлен план лечения. Исправь его, опираясь на клинические рекомендации.

### План лечения:
{}

### Рекомендации:
{}

### Исправленния:
{}"""

def formatting_prompts_func(examples):
    inputs = examples["treatment_plan"]
    context = examples["clinical_guidelines"]
    outputs = examples["corrections"]
    texts = []
    for i, c, o in zip(inputs, context, outputs):
        text = prompt_style.format(i, c, o) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

dataset = Dataset.from_list(data_list_train)
dataset = dataset.map(formatting_prompts_func, batched=True)
print(dataset[0]['text'])

In [ ]:
import pandas as pd

# Расчет статистик по размеру запросов

def count_tokens(example):
    return {"token_count": len(tokenizer.encode(example["text"]))}

dataset_with_counts = dataset.map(count_tokens)
df = dataset_with_counts.to_pandas()

print(f"Средняя длина: {df['token_count'].mean()}")
print(f"Максимальная длина: {df['token_count'].max()}")
print(f"95-й перцентиль (рекомендуемый max_seq_length): {df['token_count'].quantile(0.95)}")

In [ ]:
import gc

# Очистка кэша перед обучением
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Дообучение модели
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)
trainer.train()

In [ ]:
# Сохранение адаптеров
model.save_pretrained("medical_lora_model")
tokenizer.save_pretrained("medical_lora_model")

# Testing

In [ ]:
import os
import gdown
import json
import torch
from unsloth import FastLanguageModel
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

adapters_name = "llama_3_8b_full"
index_name = "russco_faiss"

adapters_id = "1c9N3vpGaqt7_HOIXnErj18NdJ9XCcW2i"
index_id = "18wRauo11wkQlPT6gMmy-6AlMLHWt77sG"
test_id = "12w3Eu5l8zbZrVH-9pT6WUcSc-SC9zXmK"

# Скачивание и распаковка адаптеров
gdown.download(id=adapters_id, output="adapters.zip", quiet=False)
!unzip -o -q adapters.zip -d adapters
print("Adapters downloaded and unzipped.")

# Скачивание и распаковка базы FAISS
gdown.download(id=index_id, output="index.zip", quiet=False)
!unzip -o -q index.zip -d index
print("FAISS index downloaded and unzipped.")

# Скачивание тестового датасета
gdown.download(id=test_id, output="test_dset.json", quiet=False)
with open("test_dset.json", 'r', encoding='utf-8') as file:
    test_dset = json.load(file)
print("Test dataset downloaded")

# Загрузка основной модели
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "adapters/"+adapters_name,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

# Загрузка модели эмбеддингов
model_name = "intfloat/multilingual-e5-small"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Загрузка базы данных FAISS
db = FAISS.load_local(
    "index/"+index_name,
    embeddings,
    allow_dangerous_deserialization=True
)
links_path = "index/"+index_name+"/links.json"
with open(links_path, 'r', encoding='utf-8') as file:
    links_dict = json.load(file)


def get_context(treatment_plan):
    search_query = f"query: {treatment_plan}"
    docs = db.similarity_search(search_query, k=2)

    context = "\n---\n".join([
        f"Источник ({d.metadata.get('Header 2', 'Общее')}): {d.page_content}"
        for d in docs
    ])
    sources = [d.metadata.get('source', 'RUSSCO') for d in docs]
    return context, sources

prompt_template = """Ниже представлен план лечения. Исправь его, опираясь на клинические рекомендации.
{}
### План лечения:
{}

### Рекомендации:
{}

### Исправленния:
"""

def correct_treatment_plan(treatment_plan, context, doctor=True):
    added_text = "Формулируй ответ простыми словами, которые будут понятны пациенту.\n"
    if doctor:
        added_text = ""
    final_prompt = prompt_template.format(added_text, treatment_plan, context)
    inputs = tokenizer([final_prompt], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
    new_tokens = outputs[0][inputs.input_ids.shape[-1]:]
    result = tokenizer.decode(new_tokens, skip_special_tokens=True)
    return result

In [ ]:
file_name = "_".join(["testing", adapters_name, index_name])+".txt"

result_template = """
#===================================================================================
#                                  ЗАПИСЬ {}
#===================================================================================

#============ План лечения ============
{}

#============= Контекст ===============

### Из датасета
{}

### От модели
{}
Источники:
{}

#============= Исправления ============

### Из датасета
{}

### От модели; контекст из датасета
{}

### От модели; контекст от модели
{}
"""

with open(file_name, "w") as file:

    for i, record in enumerate(test_dset):
        treatment_plan, dset_context, dset_corrections = (
            record["treatment_plan"],
            record["clinical_guidelines"],
            record["corrections"]
        )
        model_context, model_sources = get_context(treatment_plan)
        model_context_corrections = correct_treatment_plan(treatment_plan, model_context)
        dset_context_corrections  = correct_treatment_plan(treatment_plan,  dset_context)

        result = result_template.format(
            i,
            treatment_plan,
            dset_context,
            model_context,
            model_sources,
            dset_corrections,
            dset_context_corrections,
            model_context_corrections
        )
        print(result, file=file)


In [ ]:
import torch
from unsloth import FastLanguageModel

class MapReducer:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.target_input_size = 1500

    def _generate(self, prompt, text, max_new_tokens=1024):
        formatted_prompt = f"### Инструкция:\n{prompt}\n\n### Данные:\n{text}\n\n### Ответ:\n"
        inputs = self.tokenizer([formatted_prompt], return_tensors="pt").to("cuda")
        outputs = self.model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False, use_cache=True)
        return self.tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0].strip()

    def chunk_text(self, text, chunk_size=1500, overlap=200):
        tokens = self.tokenizer.encode(text)
        chunks = []
        for i in range(0, len(tokens), chunk_size - overlap):
            chunk_tokens = tokens[i : i + chunk_size]
            chunks.append(self.tokenizer.decode(chunk_tokens, skip_special_tokens=True))
            if i + chunk_size >= len(tokens): break
        return chunks

    def map_step(self, text_chunk):
        prompt = "Извлеки важные медицинские факты (диагнозы, симптомы, проведенное лечение, новые назначения) из этого фрагмента."
        return self._generate(prompt, text_chunk, max_new_tokens=500)

    def reduce_step(self, combined_summaries, is_final=False):
        if is_final:
            prompt = "На основе сжатых данных составь финальный анамнез пациента. Отдели уже проведенное лечение от планируемого."
        else:
            prompt = "Объедини и сожми следующие медицинские сводки, удаляя дубликаты и сохраняя важные клинические детали."
        return self._generate(prompt, combined_summaries, max_new_tokens=500)

    def process(self, long_history):
        chunks = self.chunk_text(long_history)
        summaries = [self.map_step(c) for c in chunks]

        current_text = "\n\n".join(summaries)
        while len(self.tokenizer.encode(current_text)) > self.target_input_size:
            new_summaries = []
            for i in range(0, len(summaries), 3):
                group = "\n\n".join(summaries[i:i+3])
                new_summaries.append(self.reduce_step(group, is_final=False))
            summaries = new_summaries
            current_text = "\n\n".join(summaries)

        return self.reduce_step(current_text, is_final=True)

reducer = MapReducer(model, tokenizer)


In [ ]:
with open("very_long_history.txt", "r") as file:
    text = file.read()

summary = reducer.process(text)
print(summary)

In [ ]:
text = "Объедини и сожми следующие медицинские сводки, удаляя дубликаты и сохраняя важные клинические детали."
len(tokenizer.encode(summary))

# OLD

In [ ]:
total_records = 22

n_correct_sources = 17
n_accurate_dset_context_corrections = -1
n_accurate_model_context_corrections = -1

import matplotlib.pyplot as plt

# Данные
categories = [
    'Правильные источники',
    'Точные исправления с источниками из датасета',
    'Точные исправления с источниками от модели',
]
values = [
    0,
    0,
    0,
]

# Настройка стиля (фиолетовая гамма)
purple_palette = ['#4B0082', '#6A5ACD', '#8A2BE2', '#9370DB', '#BA55D3']

plt.figure(figsize=(8, 6))
bars = plt.bar(categories, values, color=purple_palette)

# Добавление процентов над столбцами
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 1,
             f'{height}%', ha='center', va='bottom', fontweight='bold')

plt.title('Распределение в фиолетовых тонах', fontsize=14)
plt.ylim(0, max(values) + 10) # Запас сверху для текста
plt.show()

In [ ]:
from datasets import Dataset
import json

# Формирование промптов для дообучения

prompt_style = """Ниже представлен план лечения. Исправь его, опираясь на клинические рекомендации.

### План лечения:
{}

### Рекомендации:
{}

### Исправленния:
{}"""

def formatting_prompts_func(examples):
    inputs = examples["treatment_plan"]
    context = examples["clinical_guidelines"]
    outputs = examples["corrections"]
    texts = []
    for i, c, o in zip(inputs, context, outputs):
        text = prompt_style.format(i, c, o) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

dataset = Dataset.from_list(test_dset)
dataset = dataset.map(formatting_prompts_func, batched=True)
print(dataset[0]['text'])